# Hands-on: Hadronization - $e^+e^-$ Analysis

In this hands-on session we want to analyze the output of X-SCAPE framework simulations with $e^+e^-$ collision events. We have generated data for collisions at $\sqrt{s}=91.2$ GeV similar to the experimental runs of the ALEP experiment.

In [ ]:
# prerequisites: plots and packages
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import eig
from scipy.stats import gaussian_kde
import fastjet as fj
import sparkx.Jetscape as spxJ
import sparkx.Histogram as spxHist
import sparkx.JetAnalysis as spxJet

# put some settings for the output
width = 0.05
plotMarkerSize = 8
labelfontsize = 15
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [6., 4.5]
mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['xtick.top'] = True
mpl.rcParams['xtick.labelsize'] = 13
mpl.rcParams['xtick.major.width'] = 1.0
mpl.rcParams['xtick.minor.width'] = 0.8
mpl.rcParams['xtick.minor.visible'] = True
mpl.rcParams['xtick.direction'] = "in"
mpl.rcParams['ytick.right'] = True
mpl.rcParams['ytick.labelsize'] = 13
mpl.rcParams['ytick.major.width'] = 1.0
mpl.rcParams['ytick.minor.width'] = 0.8
mpl.rcParams['ytick.minor.visible'] = True
mpl.rcParams['ytick.direction'] = "in"
mpl.rcParams['legend.fontsize'] = 13
mpl.rcParams['legend.numpoints'] = 1
mpl.rcParams['font.size'] = 13
mpl.rcParams['savefig.format'] = "pdf"

In [ ]:
# load the data from the directory
PATH_HADRONIZATION_DATA = '../../../X-SCAPE/build/hadronization_results/hadrons_epem_final_state_hadrons.dat'

simulation_data = spxJ.Jetscape(PATH_HADRONIZATION_DATA)
full_particle_list = simulation_data.remove_particle_species(22).particle_objects_list()


## Sphericity Tensor, Sphericity and Aplanarity

We construct the second rank sphericity tensor
$$ \theta_{ij} \equiv \frac{1}{Q_p}\left| \sum\limits_a \frac{\vec{p}_{a,i}\vec{p}_{a,j}}{|\vec{p}_{a}|}\right|, $$
where the index $a$ labels the final state particles and $Q_p$ is sum of the modulus of the 3-momenta
$$Q_p\equiv \sum\limits_j |\vec{p}_j|.$$

The eigenvalues of this tensor have a geometrical meaning. First, they obey $\sum_{i=1}^3Q_i=1$.
If the event is a perfectly back-to-back jet, then two of the eigenvalues are zero.
If the event is a three-jet event, then just one of the eigenvalues is zero.
In the case of a spherical momentum distribution the eigenvalues are $Q_1=Q_2=Q_3=1/3$ [[Donoghue et. al.]](https://journals.aps.org/prd/abstract/10.1103/PhysRevD.20.2759).

The event shape is then usually defined in terms of sphericity
$$ S = \frac{3}{2}(Q_1 + Q_2) = \frac{3}{2}(1 - Q_3) $$
and aplanarity
$$ A = \frac{3}{2}Q_1, $$
where the order $Q_3 \geq Q_2\geq Q_1$ is assumed [[Ali, Kramer]](https://link.springer.com/article/10.1140/epjh/e2011-10047-1).

In [ ]:
def extract_momenta(event_hadrons):
    # 3d array (next deeper level contains events, each event contains some 3-momentum vectors of its hadrons)
    momentum_vectors = []
    energies = [] 
    # loop over the events and create the momentum vectors
    for ev in range(len(event_hadrons)):
        event_momenta = []
        for h in range(len(event_hadrons[ev])):
            event_momenta.append(np.array([event_hadrons[ev][h].px,event_hadrons[ev][h].py,event_hadrons[ev][h].pz]))
        if len(event_hadrons[ev]) > 0: # neglects the empty parts with events deleted by detector cuts
            momentum_vectors.extend([event_momenta])
    return momentum_vectors 

def Qp(p):
    norms = np.linalg.norm(p, axis=1)
    Qp = np.sum(norms)
    return Qp

def momentum_tensor(momenta):
    normalization = Qp(momenta)
    tensor = np.zeros((3, 3))
    
    for alpha in range(3):
        for beta in range(3):
            tens_alpha_beta = 0.0
            for i in range(len(momenta)):
                norm_i = np.linalg.norm(momenta[i])
                tens_alpha_beta += momenta[i][alpha] * momenta[i][beta] / norm_i
            tens_alpha_beta = np.abs(tens_alpha_beta) / normalization
            tensor[alpha][beta] = tens_alpha_beta
    
    return tensor

def compute_sphericity(eigenvalues):
    return (3./2.)*(eigenvalues[0]+eigenvalues[1])

def compute_aplanarity(eigenvalues):
    return (3./2.)*eigenvalues[0]

def compute_sphericity_tensor_quantities(momenta):
    # compute the sphericity and the aplanarity for all events
    events_sphericity = []
    events_aplanarity = []
    events_eigenvalues = []
    events_eigenvectors = []

    for ev in range(len(momenta)):
        mom_tens = momentum_tensor(momenta[ev])
        evalue, evec = eig(mom_tens)
        # sort the eigenvalues and eigenvectors in increasing order of the eigenvalues
        idx = evalue.argsort()   
        eigenValues = evalue[idx]
        eigenVectors = evec[:,idx]
    
        events_eigenvalues.extend([eigenValues])
        events_eigenvectors.extend([eigenVectors])
        events_sphericity.append(compute_sphericity(eigenValues))
        events_aplanarity.append(compute_aplanarity(eigenValues))
    return events_sphericity, events_aplanarity, events_eigenvalues, events_eigenvectors

momenta = extract_momenta(full_particle_list)
sphericity, aplanarity, eigenvalues, eigenvectors = compute_sphericity_tensor_quantities(momenta)

### The "Q-plot"

Now we generate a Q-plot to show the overall event geometry in the $e^+ e^-$-collision events.
Therefore we plot $\frac{\sqrt{3}}{2}(Q_2-Q_1)$ vs. $S$.
Events in the lower left corner are the back-to-back jet events. In the upper region along the line defined by $A$ there are planar events (three jets) and in the lower right corner one can find spherical events.

In this older [reference](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8141838/) there is also a nice Q-plot.

In [ ]:
def plot_Qplot(evals,spher):
    y_axis_quantity = []
    for ev in range(len(evals)):
        y_axis_quantity.append(np.sqrt(3)*(evals[ev][1]-evals[ev][0])/2.)

    def Qplot_triangle(S):
        return np.piecewise(S, [S < 3./4., S >= 3./4.], [lambda S: np.sqrt(3.)/3. * S, lambda S: -np.sqrt(3.)*(S-(3./4.))+(np.sqrt(3)/4.)])

    xrange = np.arange(0,1,0.0001)
    yrange = Qplot_triangle(xrange)

    # Calculate point density for the color data using Gaussian Kernel Density Estimation
    xy = np.vstack([spher, y_axis_quantity])
    density = gaussian_kde(xy)(xy)

    # Normalize the density values to [0, 1]
    density_norm = (density - np.min(density)) / (np.max(density) - np.min(density))


    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.scatter(spher, y_axis_quantity, c=density_norm, cmap='viridis', s=10, alpha=0.7, edgecolor='None')
    #ax.scatter(spher,y_axis_quantity,color='g',s=1)
    ax.plot(xrange,yrange,color='k')
    ax.set_xlabel(r"$S$")
    ax.set_ylabel(r"$\sqrt{3}(Q_2-Q_1)/2$")
    ax.set_aspect('equal')
    plt.text(0.28, 0.18, r'$Q_1=0$', dict(size=13), rotation = 30, rotation_mode = 'anchor')
    plt.text(0.88, 0.24, r'$Q_2=Q_3$', dict(size=13), rotation = -60, rotation_mode = 'anchor')
    plt.text(0.83, 0.02, r'$Q_1=Q_2$', dict(size=13), rotation = 0, rotation_mode = 'anchor')

    plt.xlim(0,None)
    plt.ylim(0,None)
    plt.tight_layout()
    plt.savefig('./Q_plot.pdf')

plot_Qplot(eigenvalues,sphericity)

### Multiplicity distribution

The next observable we want to analyze is the multiplicity of the events. We plot the probability to emmit $n$ charged hadrons $p(n) = \frac{1}{N_\mathrm{ev}}\frac{\mathrm{d}N_\mathrm{ch}}{\mathrm{d}n}$.

The cuts we will apply here are arbitrary. They do not coincide with the ones from the ALEPH collaboration. In their case there are also event cuts which depend on the sphericity axis, which we are not computing here (computationally more expensive).

In [ ]:
simulation_data = spxJ.Jetscape(PATH_HADRONIZATION_DATA)
cut_particle_list = simulation_data.remove_particle_species(22).charged_particles().pt_cut((0.2,None)).pseudorapidity_cut((-2.6,2.6)).particle_objects_list()

def compute_multiplicities(hadrons):
    multiplicity = [len(event) for event in hadrons if len(event) > 0]
    return multiplicity, len(multiplicity)

multiplicity, N_events = compute_multiplicities(cut_particle_list)

# fill histogram 
val_hist, q_bin = np.histogram(multiplicity, bins=range(1,66,2))

# statistical errors y-axis
err_hist = np.sqrt(val_hist)

# bin width x-axis
dq = (q_bin[1:]-q_bin[:-1])

# bin center x-axis
q_bin = q_bin[0:-1] + 0.5*dq

# construct y-axis spectrum (scaled) 
dN_dq = val_hist/N_events/dq

# error bars y-axis 
err_dN_dq = err_hist/N_events/dq
    
# generate plot
fig = plt.figure()
plt.errorbar(q_bin, dN_dq, fmt='s', xerr=0.5*dq, yerr=err_dN_dq, color='g')

#axes setting
plt.yscale('log')
plt.xlabel(r"$n$")
plt.ylabel(r"$p(n)$")
plt.savefig('./epem_multiplicity.pdf')

### Transverse momentum spectra

Now we want to look at some $p_\mathrm{T}$ spectra of charged hadrons, while splitting them up into hadrons from recombination and string fragmentation. In the next step we plot the same spectra for $K^{\pm}$ mesons.

In [ ]:
def create_spectrum(hadrons, pt_bins, N_ev, deta):
    hist = spxHist.Histogram(pt_bins)
    for event in range(len(hadrons)):
        for hadron in range(len(hadrons[event])):
            hist.add_value(hadrons[event][hadron].pt_abs())

    hist_bin_widths = hist.bin_width()
    hist_bin_centers = hist.bin_centers()
    err_hist = hist.statistical_error() / (N_ev * 2. * np.pi * hist_bin_centers * hist_bin_widths * deta)
    hist.scale_histogram(1./(N_ev * 2. * np.pi * hist_bin_centers * hist_bin_widths * deta))
    data_hist = hist.histogram()

    return hist_bin_centers, hist_bin_widths, data_hist, err_hist

In [ ]:
pseudorapidity_range = (-2.6,2.6)
deta = np.abs(pseudorapidity_range[1]-pseudorapidity_range[0])

def create_spectrum(hadrons, pt_bins, N_ev, deta):
    hist = spxHist.Histogram(pt_bins)
    for event in range(len(hadrons)):
        for hadron in range(len(hadrons[event])):
            hist.add_value(hadrons[event][hadron].pt_abs())

    hist_bin_widths = hist.bin_width()
    hist_bin_centers = hist.bin_centers()
    err_hist = hist.statistical_error() / (N_ev * 2. * np.pi * hist_bin_centers * hist_bin_widths * deta)
    hist.scale_histogram(1./(N_ev * 2. * np.pi * hist_bin_centers * hist_bin_widths * deta))
    data_hist = hist.histogram()

    return hist_bin_centers, hist_bin_widths, data_hist, err_hist

simulation_data1 = spxJ.Jetscape(PATH_HADRONIZATION_DATA)
recombination_hadrons = simulation_data1.remove_particle_species(22).charged_particles().pt_cut((0.1,None)).pseudorapidity_cut(pseudorapidity_range).particle_status([11,12]).particle_objects_list()
multiplicity1, N_events1 = compute_multiplicities(recombination_hadrons)

simulation_data2 = spxJ.Jetscape(PATH_HADRONIZATION_DATA)
string_frag_hadrons = simulation_data2.remove_particle_species(22).charged_particles().pt_cut((0.1,None)).pseudorapidity_cut(pseudorapidity_range).particle_status([21,22]).particle_objects_list()
multiplicity2, N_events2 = compute_multiplicities(string_frag_hadrons)

simulation_data3 = spxJ.Jetscape(PATH_HADRONIZATION_DATA)
combined_hadrons = simulation_data3.remove_particle_species(22).charged_particles().pt_cut((0.1,None)).pseudorapidity_cut(pseudorapidity_range).particle_objects_list()
multiplicity3, N_events3 = compute_multiplicities(combined_hadrons)

charged_hadron_pt = np.array([0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.5, 7.0, 9.0, 12.0])

hist_reco_bin_centers, hist_reco_bin_widths, data_hist_reco, err_hist_reco = create_spectrum(recombination_hadrons,charged_hadron_pt,N_events1,deta)
hist_frag_bin_centers, hist_frag_bin_widths, data_hist_frag, err_hist_frag = create_spectrum(string_frag_hadrons,charged_hadron_pt,N_events2,deta)
hist_combined_bin_centers, hist_combined_bin_widths, data_hist_combined, err_hist_combined = create_spectrum(combined_hadrons,charged_hadron_pt,N_events3,deta)

In [ ]:
fig, ax1 = plt.subplots()

ax1.errorbar(hist_reco_bin_centers, data_hist_reco, fmt='s', xerr=hist_reco_bin_widths/2., yerr=err_hist_reco, color='b', label=r"recombination")
ax1.errorbar(hist_frag_bin_centers, data_hist_frag, fmt='o', xerr=hist_frag_bin_widths/2., yerr=err_hist_frag, color='k', label=r"string frag.")
ax1.errorbar(hist_combined_bin_centers, data_hist_combined, fmt='x', xerr=hist_combined_bin_widths/2., yerr=err_hist_combined, color='g', label=r"all hadrons")

ax1.set_yscale('log')
ax1.legend(loc=1)
annotation_string = r"""$e^+e^-$, $\sqrt{s}=91.2$ GeV
ch. hadrons
$|\eta|\leq 2.6$, $p_\mathrm{T}\geq 0.1$ GeV
"""
ax1.annotate(annotation_string,xy=(0.05, 0.03), xycoords='axes fraction')
ax1.set_xlabel(r"$p_\mathrm{T}$ [GeV]")
ax1.set_ylabel(r"$\frac{1}{N_\mathrm{ev}}\,\frac{\mathrm{d}^2 N}{2\pi p_{\mathrm{T}}\mathrm{d}p_{\mathrm{T}}\mathrm{d}\eta}$ [1/GeV$^2$]")
plt.savefig('./epem_hadron_spectrum_charged.pdf', bbox_inches='tight')

In [ ]:
# perform the same analysis for Kaons
pseudorapidity_range = (-2.6,2.6)
deta = np.abs(pseudorapidity_range[1]-pseudorapidity_range[0])

simulation_data1 = spxJ.Jetscape(PATH_HADRONIZATION_DATA)
recombination_hadrons = simulation_data1.remove_particle_species(22).charged_particles().pt_cut((0.1,None)).pseudorapidity_cut(pseudorapidity_range).particle_status([11,12]).particle_species([321,-321]).particle_objects_list()
multiplicity1, N_events1 = compute_multiplicities(recombination_hadrons)

simulation_data2 = spxJ.Jetscape(PATH_HADRONIZATION_DATA)
string_frag_hadrons = simulation_data2.remove_particle_species(22).charged_particles().pt_cut((0.1,None)).pseudorapidity_cut(pseudorapidity_range).particle_status([21,22]).particle_species([321,-321]).particle_objects_list()
multiplicity2, N_events2 = compute_multiplicities(string_frag_hadrons)

simulation_data3 = spxJ.Jetscape(PATH_HADRONIZATION_DATA)
combined_hadrons = simulation_data3.remove_particle_species(22).charged_particles().pt_cut((0.1,None)).pseudorapidity_cut(pseudorapidity_range).particle_species([321,-321]).particle_objects_list()
multiplicity3, N_events3 = compute_multiplicities(combined_hadrons)

charged_hadron_pt = np.array([0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.5, 7.0, 9.0, 12.0])

hist_reco_bin_centers, hist_reco_bin_widths, data_hist_reco, err_hist_reco = create_spectrum(recombination_hadrons,charged_hadron_pt,N_events1,deta)
hist_frag_bin_centers, hist_frag_bin_widths, data_hist_frag, err_hist_frag = create_spectrum(string_frag_hadrons,charged_hadron_pt,N_events2,deta)
hist_combined_bin_centers, hist_combined_bin_widths, data_hist_combined, err_hist_combined = create_spectrum(combined_hadrons,charged_hadron_pt,N_events3,deta)

In [ ]:
fig, ax1 = plt.subplots()

ax1.errorbar(hist_reco_bin_centers, data_hist_reco, fmt='s', xerr=hist_reco_bin_widths/2., yerr=err_hist_reco, color='b', label=r"recombination")
ax1.errorbar(hist_frag_bin_centers, data_hist_frag, fmt='o', xerr=hist_frag_bin_widths/2., yerr=err_hist_frag, color='k', label=r"string frag.")
ax1.errorbar(hist_combined_bin_centers, data_hist_combined, fmt='x', xerr=hist_combined_bin_widths/2., yerr=err_hist_combined, color='g', label=r"all hadrons")

ax1.set_yscale('log')
ax1.legend(loc=1)
annotation_string = r"""$e^+e^-$, $\sqrt{s}=91.2$ GeV
$K^{\pm}$
$|\eta|\leq 2.6$, $p_\mathrm{T}\geq 0.1$ GeV
"""
ax1.annotate(annotation_string,xy=(0.05, 0.03), xycoords='axes fraction')
ax1.set_xlabel(r"$p_\mathrm{T}$ [GeV]")
ax1.set_ylabel(r"$\frac{1}{N_\mathrm{ev}}\,\frac{\mathrm{d}^2 N}{2\pi p_{\mathrm{T}}\mathrm{d}p_{\mathrm{T}}\mathrm{d}\eta}$ [1/GeV$^2$]")
plt.savefig('./epem_hadron_spectrum_kaon.pdf', bbox_inches='tight')

## Jet analysis

In the following part we will analyze the jet observables by using the [fastjet](https://fastjet.fr/repo/doxygen-3.4.1/) package with the anti-$k_\mathrm{T}$ algorithm in its $e^+e^-$ variant.
The considered jet radius is $R = 0.4$, as it is widely chosen for other collision systems.
The distance metric is
$$ d_{i,j}\equiv \min(E_i^{-2},E_j^{-2})\frac{1-\cos\theta_{ij}}{1-\cos R} $$
$$ d_{iB} \equiv E_i^{-2}, $$
with $d_{ij}$ the distance between two pseudojets $i$ and $j$, $E_i$ the energy and $\theta_{ij}$ the opening angle.

In [ ]:
# take the cut_particle_list and analyze it in fastjet
jets = spxJet.JetAnalysis()
jets.perform_jet_finding(cut_particle_list, jet_R=0.4, jet_eta_range=(-2.,2.), jet_pt_range=(10.,None),output_filename="epem_jets.dat",jet_algorithm=fj.ee_genkt_algorithm)

jets.read_jet_data("./epem_jets.dat")

jet_data = jets.get_jets()
associated_data = jets.get_associated_particles()

jet_energies = [jet[6] for jet in jet_data]


### Jet Energy Spectra

The jet energy spectrum is defined by 
$$\frac{1}{N_{\mathrm{ev}}} \frac{\mathrm{d}N_{\mathrm{jet}}}{\mathrm{d}E_{\mathrm{jet}}}.$$
$N_{\mathrm{ev}}$ is the number of events with hard scatterings simulated and $N_{\mathrm{jet}}$ is the total number of jets.

In [ ]:
# bin settings
E_min = 10.
E_max = 65.
E_bins = np.arange(E_min, E_max, 1.0)

# Fill Histogram
n, E = np.histogram(jet_energies, bins=E_bins)

# Statistical Errors
err_n = np.sqrt(n)

# bin width
dE = (E[1:]-E[:-1])
# bin center
E = E[0:-1] + 0.5*dE

# Jet Spectrum 
dn_dE = n/N_events/dE

# Errors 
err_dn_dE = err_n/N_events/dE

# Generate Plots
fig = plt.figure()

plt.errorbar(E, dn_dE, fmt='s', xerr=0.5*dE, yerr=err_dn_dE, color='g')

#axes setting
plt.yscale('log')
plt.xlabel(r"$E_\mathrm{jet}$ [GeV]")
plt.ylabel(r"$(1/N_{\mathrm{ev}})\mathrm{d}N_{\mathrm{jet}}/\mathrm{d}E_\mathrm{jet}$")
plt.xlim(E_min,E_max)

# save plots
plt.tight_layout()
plt.savefig('./epem_n_jet.pdf')

### Homework